In [1]:
from __future__ import unicode_literals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import glob
import json
import os
import re
import sys
import time
import traceback
from builtins import open
from time import sleep

In [ ]:
#카테고리별 url(상의, 하의, 원피스)
root_urls = ['https://www.seoulstore.com/categories/1251/regDatetime/desc',
            'https://www.seoulstore.com/categories/1253/regDatetime/desc',
            'https://www.seoulstore.com/categories/1252/regDatetime/desc']

In [ ]:
def output(data, filepath,nowtime=False):
    out = json.dumps(data, ensure_ascii=False)
    
    if nowtime==True:
        now = time.localtime()
        filepath_temp = filepath.split('.json')
        filepath = ''.join(filepath_temp[-1]) + "_%02d%02d%02d_%02d%02d%02d.json" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            f.write(out)
#        os.chmod(filepath, 0o777)        # for permission
    else:
        print(out)
        


"""
Arguments:
- urls: 리스트. 크롤링할 url들을 원소로 함.
- num: url당 크롤링할 아이템 수
- wait_time: 로딩을 몇 초까지 기다릴지

Return:
- total_posts: 리스트. 아이템 당 {"product_url": 상품url, "img_url": 이미지url} 정보를 크롤링한 것을 원소로 함.

크롤링 완료 후 "key":고유번호(5자리수)를 추가해줄 것임 (key를 사후에 추가하는 이유는 중복 크롤링 처리를 쉽게 하기 위함임)
"""

total_posts = []         # 전체 크롤링 결과를 저장할 리스트
product_set = set()   # 중복 크롤링 거르기 위한 셋. product_url을 원소로 함
unit_of_scroll = 10      # 반복할 스크롤 단위
count_nolonger = 0  # 목표 갯수 미만 & 스크롤이 더 이상 안 될 때를 대비한 카운트 변수


# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨김
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(
    '--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
# 크롬 브라우저 실행
browser = webdriver.Chrome(executable_path='chromedriver_81', chrome_options=chrome_options)
# 대기가능한 최대 시간
wait = WebDriverWait(browser, wait_time)

posts = []  # 당해 url 크롤링 결과를 저장할 리스트

browser.get(url)
body = browser.find_element_by_tag_name('body')

#카테고리 값
category = url.split('categories/')[-1].split('/')[0]
#wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'image_container')))
#category = browser.find_element_by_class_name('Category_Nav').find_elements_by_class_name('active')[0].text
    
    
# 초기에 목표값만큼 스크롤 - 갈수록 속도가 느려져서 슬립타임 세분화
count_temp = 0
for i in range(int(num // 12 * 1.5)):
    body.send_keys(Keys.END)
    if i < 40 : time.sleep(0.5)
    elif 40<=i<200 : time.sleep(1)
    else : time.sleep(2)
    count_temp += 1

# 목표 갯수 도달까지 스크롤
breakpoint = 0
while len(posts) < num and breakpoint == 0 :
    # 현재까지 리스트 개수 카운트
    if count_temp >= unit_of_scroll:
        count_temp = 0
        ele_posts = browser.find_element_by_class_name('products_container').find_elements_by_class_name('ProductItem')
        cur_posts_count = len(ele_posts)
        print(cur_posts_count)                             # FOR DEBUGGING

    
    # 목표 갯수에 도달하지 못했을 경우 더 스크롤 (단위로)
    if cur_posts_count < num :
        print('목표 갯수에 도달하지 못했을 경우 더 스크롤_start')
        last_height = browser.execute_script("return document.body.scrollHeight")
        for _ in range(unit_of_scroll):
            body.send_keys(Keys.END);     time.sleep(1)
            count_temp += 1
        new_height = browser.execute_script("return document.body.scrollHeight")
    
        #목표 갯수 미만 & 스크롤이 더 이상 안 될 때
        if last_height == new_height :
            body.send_keys(Keys.PAGE_UP);     time.sleep(2)
            body.send_keys(Keys.END);             time.sleep(5)
            count_nolonger += 1
        else : count_nolonger = 0
        if count_nolonger > 10 : breakpoint = 1
        print('목표 갯수에 도달하지 못했을 경우 더 스크롤_end')
    
    #목표에 도달 시
    else : breakpoint = 1; print('목표에 도달 함')

# 저장 부분
posts = []
for ele in ele_posts :
    image_container = ele.find_element_by_class_name('image_container')
    info_container = ele.find_element_by_class_name('info_container')
    product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
    
    dict_post = {"product_url": product_url}
#    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
    ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
    dict_post["img_url"] = ele_img.get_attribute("src")
    dict_post['category'] = category
    posts.append(dict_post)
output(posts, 'tong_3000_',True)